# QAOA gridsearch

In [3]:
#Initialisation cell
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from docplex.mp.model import Model
from qiskit.providers.ibmq.managed import IBMQJobManager

from qiskit import BasicAer, Aer
from qiskit.aqua.algorithms import VQE, ExactEigensolver,NumPyEigensolver,QAOA
from qiskit.aqua.components.optimizers import SPSA, COBYLA,ADAM, L_BFGS_B
from qiskit.circuit.library import RealAmplitudes #(uses CX entangling) 
from qiskit.optimization.applications.ising.common import sample_most_likely
from qiskit.circuit.library import TwoLocal
#from qiskit.aqua.components.variational_forms import RY
from qiskit.aqua import QuantumInstance

from qiskit.optimization import QuadraticProgram
from qiskit.tools.visualization import plot_histogram
from qiskit.optimization.applications.ising import docplex
from qiskit.optimization.algorithms import MinimumEigenOptimizer,RecursiveMinimumEigenOptimizer

# setup aqua logging
import logging
from qiskit.aqua import set_qiskit_aqua_logging
# set_qiskit_aqua_logging(logging.DEBUG)  # choose INFO, DEBUG to see the log

# useful additional packages 
import matplotlib.pyplot as plt
import matplotlib.axes as axes
%matplotlib inline
import numpy as np
import networkx as nx
import pandas as pd
from   matplotlib import cm
from   matplotlib.ticker import LinearLocator, FormatStrFormatter
%config InlineBackend.figure_format = 'svg' # Makes the images look nice

# importing Qiskit
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute
from qiskit.providers.ibmq      import least_busy
from qiskit.tools.monitor       import job_monitor
from qiskit.providers.aer import QasmSimulator

Important prelim functions

In [ ]:
def CSVtoNumpyArray(rawdata):
    """
    Input: 
    rawdata = a csv file (insert name as a string)

    Output:
    two numpy matrices in a tuple
    """
    data = pd.read_csv(rawdata)  #Reads the data in as a pandas object
    c = data.columns
    column = int(c[0])
    final_data1 = data.iloc[:column,:].values  #Sets data into a series of numpy arrays of strings
    final_data2 = data.iloc[column:,:].values  #1 is for the first matrix(loc) and 2 is for the second(flow)
    

    #Forms the matrix as a numpy array (easier to work with) instead of an list of lists of strings
    def string_to_integers(final_data):
        matrix = []
        for j in range(column):
            string = final_data[j][0]
            string2 = string.split(" ")
            emptyarray = []
            for i in string2:
                if i != '':
                    emptyarray.append(int(i))
            matrix.append(emptyarray)
        npmatrix = np.array(matrix) 
        return npmatrix
    return string_to_integers(final_data1),string_to_integers(final_data2)

In [ ]:
def qap_value(z, MatrixLoc, MatrixFlow):
    """Compute the TSP value of a solution.
    Args:
        z (list[int]): list of allocations
        MatrixLoc (numpy array): matrix of distances
        MatrixFlow (numpy array): matrix of flow
    Returns:
        float: value of the QAP
    """
    matrix_length = len(MatrixLoc)
    x = np.reshape(z, (matrix_length,matrix_length))
    
    total = 0
    for i in range(matrix_length):
        for j in range(matrix_length):
            for k in range(matrix_length):
                for p in range(matrix_length):
                        total += MatrixLoc[i,j]* MatrixFlow[k,p]*x[i,k]*x[j,p]
    
    return total

Feasibility functions

In [ ]:
def qap_feasible(x):
    """Check whether a solution is feasible or not.

    Args:
        x (numpy.ndarray) : binary string as numpy array.

    Returns:
        bool: feasible or not.
    """
    n = int(np.sqrt(len(x)))
    y = np.reshape(x, (n,n))
   
    for i in range(n):
        if sum(y[i, p] for p in range(n)) != 1:
            return False
    for p__ in range(n):
        if sum(y[i, p__] for i in range(n)) != 1:
            return False
    return True

In [ ]:
def choose_best_feasible(eigenstates):
    """
    Input:
    eigenstates = dictionary
    
    Output:
    feasible binary 1D numpyarray
    probability of this answer
    
    """
    bestinarray = sorted(eigenstates.items(), key=lambda item: item[1])[::-1]
    feasible = False
    counter = 0
    total = sum(eigenstates.values())
    
    feasible=False
    while feasible==False and counter<len(bestinarray):
        #string to array
        bestasint = np.array([0])
        for i in bestinarray[counter][0]:
            bestasint = np.hstack((bestasint, int(i)))
        feasible = qap_feasible(bestasint[1:])
        frequency = bestinarray[counter][1]
        counter += 1
        
    if feasible == False:
        return feasible
    else:
        return bestasint[1:], frequency/total

## The QAOA function

In [ ]:
def testing_quantum(ins, start, stop, step):
    """
    Input:
    ins: name of instance
    start
    stop
    step
    
    Output:
    outmatrix : [soln,result time,eigenvector,number of spsa trials]
    """
    
    #get matrix
    datamatrix = CSVtoNumpyArray(ins)
    MatrixLoc = datamatrix[0]
    MatrixFlow = datamatrix[1]
    n = len(MatrixLoc)

    # Create an instance of a model and variables.
    thename = "qap" + str(n)
    mdl = Model(name=thename)
    x = {(i,p): mdl.binary_var(name='x_{0}_{1}'.format(i,p)) for i in range(n) for p in range(n)}

    # Object function
    qap_func = mdl.sum(MatrixLoc[i,j]* MatrixFlow[k,p]*x[i,k]*x[j,p] for i in range(n) for j in range(n) for p in range(n) for k in range(n))
    mdl.minimize(qap_func)

    # Constraints
    for i in range(n):
        mdl.add_constraint(mdl.sum(x[(i,p)] for p in range(n)) == 1)
    for p in range(n):
        mdl.add_constraint(mdl.sum(x[(i,p)] for i in range(n)) == 1)
    print(mdl.export_to_string())
    qubitOp_docplex, offset_docplex = docplex.get_operator(mdl) #potential error

    #QAOA
    machinesolutions = []
    rmachinesolutions = []
    seed = 10598
    mod = QuadraticProgram()
    mod.from_docplex(mdl)
    #print(mod.export_as_lp_string())

    for i in range(start,stop,step):
        spsa = SPSA(maxiter=i)
        backend = Aer.get_backend('qasm_simulator')
        backend.set_options(method= "matrix_product_state")
        quantum_instance = QuantumInstance(backend=backend,seed_simulator=seed,
                                           seed_transpiler=seed, skip_qobj_validation = False, shots = 8192)

        qaoa = QAOA(qubitOp_docplex,optimizer=spsa, p=3,include_custom=True)
        result = qaoa.run(quantum_instance)
        print(result['optimal_point'])
        #print('QAOA vector:', result['eigenstate'])
        print('QAOA time:', result['optimizer_time'])
        print('QAOA QAP objective:', result['eigenvalue'] + offset_docplex)
        x = choose_best_feasible(result['eigenstate'])
        print(x,i)
        if x == False:
            soln = 'No solution' #note fails if false
        else:
            soln = qap_value(x[0],MatrixFlow,MatrixLoc) #note fails if false
        print(soln)
        machinesolutions.append([soln, result['optimizer_time'],x,i])
    print(machinesolutions)
    return machinesolutions
    

In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG) # log the steps of the algorithm and results
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
ins = "made5.csv"
start = 1
stop = 5
step = 1
ans = testing_quantum(ins, start, stop, step)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap5

Minimize
 obj: [ 400 x_0_0*x_1_2 + 600 x_0_0*x_1_4 + 400 x_0_0*x_2_2 + 600 x_0_0*x_2_4
      + 752 x_0_0*x_3_2 + 1128 x_0_0*x_3_4 + 400 x_0_0*x_4_2 + 600 x_0_0*x_4_4
      + 600 x_0_1*x_1_3 + 600 x_0_1*x_2_3 + 1128 x_0_1*x_3_3 + 600 x_0_1*x_4_3
      + 400 x_0_2*x_1_0 + 400 x_0_2*x_2_0 + 752 x_0_2*x_3_0 + 400 x_0_2*x_4_0
      + 600 x_0_3*x_1_1 + 200 x_0_3*x_1_4 + 600 x_0_3*x_2_1 + 200 x_0_3*x_2_4
      + 1128 x_0_3*x_3_1 + 376 x_0_3*x_3_4 + 600 x_0_3*x_4_1 + 200 x_0_3*x_4_4
      + 600 x_0_4*x_1_0 + 200 x_0_4*x_1_3 + 600 x_0_4*x_2_0 + 200 x_0_4*x_2_3
      + 1128 x_0_4*x_3_0 + 376 x_0_4*x_3_3 + 600 x_0_4*x_4_0 + 200 x_0_4*x_4_3
      + 176 x_1_0*x_2_2 + 264 x_1_0*x_2_4 + 400 x_1_0*x_3_2 + 600 x_1_0*x_3_4
      + 288 x_1_0*x_4_2 + 432 x_1_0*x_4_4 + 264 x_1_1*x_2_3 + 600 x_1_1*x_3_3
      + 432 x_1_1*x_4_3 + 176 x_1_2*x_2_0 + 400 x_1_2*x_3_0 + 288 x_1_2*x_4_0
      + 264 x_1_3*x_2_1 + 88 x_1_3*x_2_4 + 6

INFO:qiskit.compiler.assemble:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assemble:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assemble:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assemble:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assemble:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assemble:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assemble:Total Assembly Time - 0.99945 (ms)
INFO:qiskit.compiler.assemble:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assemble:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assemble:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.aqua.quantum_instance:
Qiskit Terra version: 0.16.2
Backend: 'qasm_simulator (AerProvider)', with following setting:
{'basis_gates': ['ccx', 'cp', 'cx', 'cz', 'delay', 'h', 'id', 'kraus', 'p', 'roerror', 's', 'sdg', 'snapshot', 'swap', 'sx', 't', 'tdg', 'u', 'u1', 'u2', 'u3', 'unitary', 'x', 'y', 'z'], 'coupling_map': None}
{'initial_la

INFO:qiskit.aqua.algorithms.vq_algorithm:Starting optimizer.
bounds=[(0, 3.141592653589793), (0, 3.141592653589793), (0, 3.141592653589793), (0, 6.283185307179586), (0, 6.283185307179586), (0, 6.283185307179586)]
initial point=[0, 0, 0, 0, 0, 0]
DEBUG:qiskit.aqua.components.optimizers.optimizer:WARNING: SPSA does not support bounds. It will be ignored.
DEBUG:qiskit.aqua.components.optimizers.spsa:Parameters: [0.62831853 0.1        0.602      0.101      0.        ]
DEBUG:qiskit.aqua.components.optimizers.spsa:Calibration...
DEBUG:qiskit.aqua.components.optimizers.spsa:calibration step # 0 of 1
INFO:qiskit.transpiler.runningpassmanager:Pass: UnrollCustomDefinitions - 5.99527 (ms)
INFO:qiskit.transpiler.passes.basis.basis_translator:Begin BasisTranslator from source basis {('snapshot', 25), ('measure', 1), ('rz', 1), ('cx', 2), ('h', 1), ('rx', 1)} to target basis {'u3', 'unitary', 'roerror', 't', 'swap', 'u1', 'tdg', 'cz', 'h', 'u', 'z', 'u2', 'ccx', 'sx', 'reset', 'y', 'x', 'cp', 'measu

In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG) # log the steps of the algorithm and results
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
ins = "made9.csv"
start = 1
stop = 2
step = 1
ans = testing_quantum(ins, start, stop, step)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap9

Minimize
 obj: [ 256 x_0_0*x_1_1 + 512 x_0_0*x_1_2 + 256 x_0_0*x_1_6 + 544 x_0_0*x_2_1
      + 1088 x_0_0*x_2_2 + 544 x_0_0*x_2_6 + 776 x_0_0*x_3_1 + 1552 x_0_0*x_3_2
      + 776 x_0_0*x_3_6 + 600 x_0_0*x_4_1 + 1200 x_0_0*x_4_2 + 600 x_0_0*x_4_6
      + 560 x_0_0*x_5_1 + 1120 x_0_0*x_5_2 + 560 x_0_0*x_5_6 + 600 x_0_0*x_6_1
      + 1200 x_0_0*x_6_2 + 600 x_0_0*x_6_6 + 320 x_0_0*x_7_1 + 640 x_0_0*x_7_2
      + 320 x_0_0*x_7_6 + 192 x_0_0*x_8_1 + 384 x_0_0*x_8_2 + 192 x_0_0*x_8_6
      + 256 x_0_1*x_1_0 + 384 x_0_1*x_1_2 + 128 x_0_1*x_1_3 + 128 x_0_1*x_1_5
      + 256 x_0_1*x_1_8 + 544 x_0_1*x_2_0 + 816 x_0_1*x_2_2 + 272 x_0_1*x_2_3
      + 272 x_0_1*x_2_5 + 544 x_0_1*x_2_8 + 776 x_0_1*x_3_0 + 1164 x_0_1*x_3_2
      + 388 x_0_1*x_3_3 + 388 x_0_1*x_3_5 + 776 x_0_1*x_3_8 + 600 x_0_1*x_4_0
      + 900 x_0_1*x_4_2 + 300 x_0_1*x_4_3 + 300 x_0_1*x_4_5 + 600 x_0_1*x_4_8
      + 560 x_0_1*x_5_0 + 840 x_0_1*x_5_2 

INFO:qiskit.compiler.assemble:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assemble:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assemble:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assemble:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assemble:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assemble:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assemble:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assemble:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assemble:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assemble:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.aqua.quantum_instance:
Qiskit Terra version: 0.16.2
Backend: 'qasm_simulator (AerProvider)', with following setting:
{'basis_gates': ['ccx', 'cp', 'cx', 'cz', 'delay', 'h', 'id', 'kraus', 'p', 'roerror', 's', 'sdg', 'snapshot', 'swap', 'sx', 't', 'tdg', 'u', 'u1', 'u2', 'u3', 'unitary', 'x', 'y', 'z'], 'coupling_map': None}
{'initial_la

INFO:qiskit.aqua.algorithms.vq_algorithm:Starting optimizer.
bounds=[(0, 3.141592653589793), (0, 3.141592653589793), (0, 3.141592653589793), (0, 6.283185307179586), (0, 6.283185307179586), (0, 6.283185307179586)]
initial point=[0, 0, 0, 0, 0, 0]
DEBUG:qiskit.aqua.components.optimizers.optimizer:WARNING: SPSA does not support bounds. It will be ignored.
DEBUG:qiskit.aqua.components.optimizers.spsa:Parameters: [0.62831853 0.1        0.602      0.101      0.        ]
DEBUG:qiskit.aqua.components.optimizers.spsa:Calibration...
DEBUG:qiskit.aqua.components.optimizers.spsa:calibration step # 0 of 1
INFO:qiskit.transpiler.runningpassmanager:Pass: UnrollCustomDefinitions - 60.96506 (ms)
INFO:qiskit.transpiler.passes.basis.basis_translator:Begin BasisTranslator from source basis {('rz', 1), ('measure', 1), ('snapshot', 81), ('cx', 2), ('h', 1), ('rx', 1)} to target basis {'unitary', 'kraus', 'ccx', 'snapshot', 'reset', 'u', 'u1', 'barrier', 'p', 'u3', 'u2', 'measure', 'cx', 'sx', 'cp', 'id', 't

DEBUG:qiskit.transpiler.passes.basis.basis_translator:Updating transform for mapped instr ('rz', 1) rz, ['rz[0]'] from 
      ┌───────────┐
q0_0: ┤ rz(rz[0]) ├
      └───────────┘
DEBUG:qiskit.transpiler.passes.basis.basis_translator:Updated transform for mapped instr ('rz', 1) rz, ['rz[0]'] to
global phase: -0.5*rz[0]
      ┌───────────┐
q0_0: ┤ U1(rz[0]) ├
      └───────────┘
DEBUG:qiskit.transpiler.passes.basis.basis_translator:Composing transform step: h/1 [] =>
     ┌─────────┐
q_0: ┤ U2(0,π) ├
     └─────────┘
DEBUG:qiskit.transpiler.passes.basis.basis_translator:Updating transform for mapped instr ('h', 1) h, [] from 
      ┌───┐
q4_0: ┤ h ├
      └───┘
DEBUG:qiskit.transpiler.passes.basis.basis_translator:Updated transform for mapped instr ('h', 1) h, [] to
      ┌─────────┐
q4_0: ┤ U2(0,π) ├
      └─────────┘
DEBUG:qiskit.transpiler.passes.basis.basis_translator:Composing transform step: cx/2 [] =>
global phase: 3π/4
     ┌───┐     ┌────────┐┌───┐     ┌────────┐┌───┐┌───┐     

DEBUG:qiskit.transpiler.passes.basis.basis_translator:Updated transform for mapped instr ('rx', 1) rx, ['rx[0]'] to
      ┌────────────┐
q5_0: ┤ R(rx[0],0) ├
      └────────────┘
DEBUG:qiskit.transpiler.passes.basis.basis_translator:Composing transform step: r/1 [Parameter(theta), Parameter(phi)] =>
     ┌───────────────────────────────────────────────────────────┐
q_0: ┤ U3(theta,phi - 1.5707963267949,1.5707963267949 - 1.0*phi) ├
     └───────────────────────────────────────────────────────────┘
DEBUG:qiskit.transpiler.passes.basis.basis_translator:Updating transform for mapped instr ('rx', 1) rx, ['rx[0]'] from 
      ┌────────────┐
q5_0: ┤ R(rx[0],0) ├
      └────────────┘
DEBUG:qiskit.transpiler.passes.basis.basis_translator:Updated transform for mapped instr ('rx', 1) rx, ['rx[0]'] to
      ┌────────────────────┐
q5_0: ┤ U3(rx[0],-π/2,π/2) ├
      └────────────────────┘
INFO:qiskit.transpiler.passes.basis.basis_translator:Basis translation paths composed in 0.224s.
INFO:qiskit.tra

In [2]:
import qiskit
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
qiskit.__qiskit_version__

{'qiskit-terra': '0.17.4', 'qiskit-aer': '0.8.2', 'qiskit-ignis': '0.6.0', 'qiskit-ibmq-provider': '0.14.0', 'qiskit-aqua': '0.9.2', 'qiskit': '0.27.0', 'qiskit-nature': '0.1.3', 'qiskit-finance': None, 'qiskit-optimization': '0.1.0', 'qiskit-machine-learning': None}